# Multi-Competitor NER & Sentiment Analysis - IMPROVED SENTIMENT MODEL

**Performance Improvements Based on Feedback:**
- NER Model: **90.2% F1** ✅ (Keep as is - excellent performance)
- Sentiment Model: **58.0% F1** → Target: **>70% F1** 🎯

**Sentiment Model Improvements:**
1. ✅ **Data Augmentation** - Expand 212 training samples with synonym replacement, back-translation
2. ✅ **Focal Loss** - Better handle class imbalance, focus on hard examples
3. ✅ **Optimized Hyperparameters** - Lower LR (1e-5), more epochs (10), better warmup (0.2)
4. ✅ **Early Stopping** - Stop when val F1 plateaus (patience=3)
5. ✅ **Better Contextualization** - Improved prompt engineering
6. ✅ **Detailed Analysis** - Confusion matrix, per-class metrics, error analysis

**Expected Performance:**
- NER F1: **>0.90** (maintained)
- Sentiment F1: **>0.70** (improved from 0.58)

## 1. Setup & Installation

In [ ]:
# Install required libraries
!pip install -q transformers datasets torch torchvision accelerate
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q sentencepiece protobuf
!pip install -q openpyxl xlsxwriter
!pip install -q chardet
!pip install -q nltk  # For data augmentation
!pip install -q nlpaug  # Advanced augmentation

# Download NLTK data for augmentation
import nltk
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
print("✓ NLTK data downloaded for augmentation")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import re
import gc
import warnings
import chardet
from pathlib import Path
from copy import deepcopy
import random
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup
)

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, f1_score, precision_recall_fscore_support
)
from sklearn.utils.class_weight import compute_class_weight

# Augmentation
import nlpaug.augmenter.word as naw
from nltk.corpus import wordnet

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Set seeds
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("✓ Libraries imported successfully!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Using AdamW from: torch.optim")

In [ ]:
# Check GPU and configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    gpu_memory_gb = gpu_props.total_memory / 1e9
    print(f"GPU: {gpu_props.name}")
    print(f"GPU Memory: {gpu_memory_gb:.2f} GB")
    
    # Adaptive batch size
    if gpu_memory_gb >= 15:
        BATCH_SIZE = 16
        GRAD_ACCUM_STEPS = 2
    else:
        BATCH_SIZE = 8
        GRAD_ACCUM_STEPS = 4
else:
    BATCH_SIZE = 4
    GRAD_ACCUM_STEPS = 8

EFFECTIVE_BATCH_SIZE = BATCH_SIZE * GRAD_ACCUM_STEPS
print(f"\nBatch size: {BATCH_SIZE}, Accumulation: {GRAD_ACCUM_STEPS}, Effective: {EFFECTIVE_BATCH_SIZE}")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create directories
!mkdir -p '/content/drive/MyDrive/KFC_ML_Models'
!mkdir -p '/content/results'

MODEL_SAVE_DIR = '/content/drive/MyDrive/KFC_ML_Models'
RESULTS_DIR = '/content/results'

## 2. Configuration

In [ ]:
# Competitor list (14 total)
COMPETITORS = [
    'Burger King', 'Deliveroo', "Domino's", 'Five Guys', 'Greggs',
    'Just Eat', 'KFC', "McDonald's", "Nando's", "Papa John's",
    'Pizza Hut', 'Pret a Manger', 'Taco Bell', 'Uber Eats'
]

SENTIMENT_MAP = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Model configs
NER_MODEL_NAME = 'bert-base-uncased'
SENTIMENT_MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

# Hyperparameters
MAX_SEQ_LENGTH = 128

# NER hyperparameters (keep same - performing well)
NER_LEARNING_RATE = 2e-5
NER_EPOCHS = 5
NER_WARMUP_RATIO = 0.1

# IMPROVED Sentiment hyperparameters
SENTIMENT_LEARNING_RATE = 1e-5  # Lower LR for better fine-tuning
SENTIMENT_EPOCHS = 10  # More epochs
SENTIMENT_WARMUP_RATIO = 0.2  # More warmup
SENTIMENT_EARLY_STOP_PATIENCE = 3  # Early stopping

# Data Augmentation settings
AUGMENTATION_FACTOR = 3  # Create 3x more training data

# Other
WEIGHT_DECAY = 0.01
FOCAL_LOSS_GAMMA = 2.0  # Focal loss parameter

print(f"✓ Configuration loaded")
print(f"  Competitors: {len(COMPETITORS)}")
print(f"  NER Model: {NER_MODEL_NAME}")
print(f"  Sentiment Model: {SENTIMENT_MODEL_NAME}")
print(f"\n🎯 SENTIMENT MODEL IMPROVEMENTS:")
print(f"  Learning Rate: {SENTIMENT_LEARNING_RATE} (was 2e-5)")
print(f"  Epochs: {SENTIMENT_EPOCHS} (was 5)")
print(f"  Warmup Ratio: {SENTIMENT_WARMUP_RATIO} (was 0.1)")
print(f"  Augmentation Factor: {AUGMENTATION_FACTOR}x")
print(f"  Early Stopping: Patience {SENTIMENT_EARLY_STOP_PATIENCE}")
print(f"  Focal Loss Gamma: {FOCAL_LOSS_GAMMA}")